<a href="https://colab.research.google.com/github/patcon/valency-anndata/blob/main/example-usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet \
  git+https://github.com/patcon/polis-client \
  git+https://github.com/patcon/valency-anndata

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 96.7 MB/s eta 0:00:00


In [27]:
# Customize Jupyter notebook environment settings.
import pandas as pd
pd.options.display.max_rows = 10

In [26]:
from IPython.display import Image, display

# When we are executing prospective functionality that hasn't yet been written,
# we wrap it in a conditional and in the meantime output a placeholder/mockup.
#
# Thanks to Gregor Martynus (gr2m) for concept: https://github.com/gr2m/dreamcode.io
def does_dreamcode_exist(placeholder_image=None):
  if placeholder_image:
    display(Image(url=placeholder_image, width=500))
  # Dreamcode never exists yet.
  return False

In [ ]:
import valency_anndata as val

adata = val.datasets.polis(source="https://pol.is/report/r2dfw8eambusb8buvecjt")
# These workl similarly to the above source:
#   - r2dfw8eambusb8buvecjt
#   - https://pol.is/6jrufhr6dp
#   - 6jrufhr6dp
#
# NOTE: Technically, there are some differences between
# 1. data fetched from CSV exports (best source when we have report_id) and
# 2. data fetched from API endpoints (best we can do when we only have conversation_id)

adata

AnnData object with n_obs × n_vars = 0 × 0
    uns: 'votes_csv', 'statements'

In [ ]:
val.preprocessing.rebuild_vote_matrix(adata)

adata

AnnData object with n_obs × n_vars = 65 × 43
    uns: 'votes_csv', 'statements'

In [ ]:
val.preprocessing.rebuild_vote_matrix(adata, trim_rule=0.8)

adata

AnnData object with n_obs × n_vars = 52 × 40
    uns: 'votes_csv', 'statements'

In [ ]:
adata.uns["votes_csv"]

,timestamp,datetime,comment-id,voter-id,vote
0,1544544810,Tue Dec 11 2018 16:13:30 GMT+0000 (Coordinated...,0,0,1
1,1544544823,Tue Dec 11 2018 16:13:43 GMT+0000 (Coordinated...,1,0,1
2,1544557247,Tue Dec 11 2018 19:40:47 GMT+0000 (Coordinated...,1,1,1
17,1544557306,Tue Dec 11 2018 19:41:46 GMT+0000 (Coordinated...,2,1,1
61,1544557393,Tue Dec 11 2018 19:43:13 GMT+0000 (Coordinated...,3,1,1
...,...,...,...,...,...
1487,1548327005,Thu Jan 24 2019 10:50:05 GMT+0000 (Coordinated...,41,63,1
1056,1548327011,Thu Jan 24 2019 10:50:11 GMT+0000 (Coordinated...,25,63,-1
985,1548327013,Thu Jan 24 2019 10:50:13 GMT+0000 (Coordinated...,23,63,1
1499,1665662071,Thu Oct 13 2022 11:54:31 GMT+0000 (Coordinated...,42,64,1


In [ ]:
adata.uns["statements"]

,conversation_id,txt,tid,created,quote_src_url,is_seed,is_meta,lang,pid,velocity,mod,active,agree_count,disagree_count,pass_count,count
0,6jrufhr6dp,I feel Blockchain technologies have been over-...,1,1544544823605,None,False,False,en,0,1,-1,True,12,2,1,15
1,6jrufhr6dp,"""Big data"" is no longer a hot topic.",3,1544557393305,None,False,False,en,1,1,0,True,13,20,12,45
2,6jrufhr6dp,It became clear that the business model preval...,26,1544594128774,None,False,False,en,17,1,0,True,21,5,9,35
3,6jrufhr6dp,Swing Left's campaign in waiting: building gra...,38,1544659444362,None,False,False,en,35,1,0,True,7,9,8,24
4,6jrufhr6dp,Blockchain technologies being pitched as the s...,31,1544618958734,None,False,False,en,9,1,-1,True,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,6jrufhr6dp,The seeds were laid to see the banning of end-...,18,1544565378085,None,False,False,en,4,1,0,True,12,15,14,41
39,6jrufhr6dp,The widening of social credit scoring in China...,19,1544565745966,None,False,False,en,4,1,0,True,24,9,11,44
40,6jrufhr6dp,For-profit tech companies will always be incen...,41,1544824910554,None,False,False,en,52,1,0,True,9,0,2,11
41,6jrufhr6dp,"Social media facilitated the growth of ""illibe...",29,1544594443493,None,False,False,en,17,1,1,True,25,7,4,36


In [28]:
if does_dreamcode_exist('https://imgur.com/OmmUxMo.png'):
  val.viz.langevitour("foo")
